<img src="https://i.imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px;"/>


# DSI-SG-42

## Project Two - Regression Challenge (HDB Price)

---


# 2.1 Feature Selection

In [1]:
# import libraries
import pandas as pd
import math

# setting displays
%matplotlib inline
pd.set_option('display.width', 100000)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


# import cleaned data
hdb_cleaned = pd.read_csv('../datasets/hdb_cleaned.csv', low_memory=False, index_col=0)

print(hdb_cleaned.head())

       id tranc_yearmonth             town  flat_type block       street_name storey_range  floor_area_sqm  flat_model  lease_commence_date  resale_price  tranc_year  tranc_month  mid_storey  lower  upper  mid       full_flat_type               address  price_per_sqft  hdb_age  max_floor_lvl  year_completed  commercial  market_hawker  multistorey_carpark  precinct_pavilion  total_dwelling_units  one_room_unit  two_room_unit  three_room_unit  four_room_unit  five_room_unit  exec_unit  multigen_unit  studio_apartment  one_room_rental  two_room_rental  three_room_rental  other_room_rental  postal  latitude   longitude planning_area  mall_nearest_distance  mall_within_500m  mall_within_1km  mall_within_2km  hawker_nearest_distance  hawker_within_500m  hawker_within_1km  hawker_within_2km  hawker_food_stalls  hawker_market_stalls  mrt_nearest_distance     mrt_name  bus_interchange  mrt_interchange  mrt_latitude  mrt_longitude  bus_stop_nearest_distance             bus_stop_name  bus_stop_la

#### Feature selection 

Independent variables will be dropped systematically and they have been deemed to have multicollinearity or not to be used in this analysis.

In [2]:
# We will be dropping columns that are addresses or repeated variables of another name.
# We assume that the rental units in the block will not be affecting the resale price of a unit in the same block. 

hdb_df = hdb_cleaned.drop(columns=['tranc_yearmonth', 'town', 'lease_commence_date', 'mid_storey', 'lower', 'upper', 'latitude', 'longitude',
                                'full_flat_type', 'address', 'year_completed', 'postal', 'block','price_per_sqft', 'street_name', 'storey_range',
                                'one_room_rental',  'two_room_rental',  'three_room_rental',  'other_room_rental'])


# Visual inspection of the first 5 rows of the df
#print(hdb_df.head())  # debug

In [3]:
print(hdb_cleaned['flat_model'].value_counts())

flat_model
Model A                   46261
Improved                  38514
New Generation            23366
Premium Apartment         15091
Simplified                 7239
Apartment                  6080
Standard                   4867
Maisonette                 4364
Model A2                   2064
DBSS                       1658
Model A-Maisonette          262
Adjoined flat               260
Type S1                     242
Type S2                     127
Terrace                      95
Multi Generation             56
Premium Apartment Loft       50
Improved-Maisonette          19
Premium Maisonette           15
2-room                        4
Name: count, dtype: int64


In [4]:
# Next we will be dropping the all boolean variables except multistorey_carpark, as it is an improvement from the open air carpark decades ago. 

hdb_df = hdb_df.drop(columns=['commercial', 'market_hawker', 'hawker_market_stalls', 'precinct_pavilion', 'cutoff_point', 'affiliation', 'vacancy'])

# Visual inspection of the first 5 rows of the df
#print(hdb_df.head()) # debug

In [5]:
# Remove all most variables related to school except pri_sch_name
# it has been selected for feature engineering

hdb_df = hdb_df.drop(columns = ['pri_sch_nearest_distance', 'pri_sch_affiliation', 'pri_sch_latitude',
                                'pri_sch_longitude', 'sec_sch_name', 'sec_sch_latitude', 'sec_sch_longitude', 'sec_sch_nearest_dist'])

# Visual inspection of the first 5 rows of the df
#print(hdb_df.head()) #debug

In [6]:
# Remove most variables from the bus/mrt related columns

hdb_df = hdb_df.drop(columns = ['mrt_latitude', 'mrt_longitude',  'bus_stop_name', 'bus_stop_latitude', 'bus_stop_longitude'])

# Visual inspection of the first 5 rows of the df
print(hdb_df.head())

       id  flat_type  floor_area_sqm  flat_model  resale_price  tranc_year  tranc_month  mid  hdb_age  max_floor_lvl  multistorey_carpark  total_dwelling_units  one_room_unit  two_room_unit  three_room_unit  four_room_unit  five_room_unit  exec_unit  multigen_unit  studio_apartment planning_area  mall_nearest_distance  mall_within_500m  mall_within_1km  mall_within_2km  hawker_nearest_distance  hawker_within_500m  hawker_within_1km  hawker_within_2km  hawker_food_stalls  mrt_nearest_distance     mrt_name  bus_interchange  mrt_interchange  bus_stop_nearest_distance                           pri_sch_name
0   88471     4 ROOM            90.0     Model A      680000.0        2016            5   11       15             25                    0                   142              0              0                0              96              46          0              0                 0       Kallang            1094.090418                 0                0                7               154.

# 2.2 Feature Engineering

We have selected a few independent variables that we will be engineering. 

- First, we identified that it is more likely young parents would prefer to reside near the primary school of choice as the child is still young to be considered taking public transport independently. Thus, we will create a new column with boolean values if there the school is one of the top 10 primary school in the list.

note: there were schools such as Anglo-Chinese School (Primary) and Raffles Girls' Primary School that were not in the list and were excluded even though they were rated in the top 10. Thus, schools that are ranked 11th and 12th were included instead.

In [7]:
# Identify Top Primary Schools from https://schlah.com/primary-schools

# define the list of top 10 schools
top_pri = ['Rosyth School', 'Nan Hua Primary School', 'Saint Hilda\'s Primary School', 'Catholic High School', 'Henry Park Primary School', 
               'Nanyang Primary School', 'Tao Nan School', 'Pei Hwa Presbyterian Primary School', 'CHIJ Saint Nicholas Girls\' School', 'Kong Hwa School']


top_pri = [x.lower() for x in top_pri] # lower case values
top_pri = [x.replace("'", '') for x in top_pri] # remove quotation marks
top_pri = [x.strip() for x in top_pri] # remove leading and trailing whitespaces
# print(top_pri) # check if changes have been made

# lower case school names in pri_sch_names
hdb_df['pri_sch_name'] = hdb_df['pri_sch_name'].str.lower()


# create a new binary list if resale unit has top school nearby
top_school = []

for school in hdb_df['pri_sch_name']:
    if any(school in top for top in top_pri):
        top_school.append(1)
    else:
        top_school.append(0)

hdb_df['nearby_top_sch'] = top_school # insert new list to df

#print(hdb_df.head()) # debug


- Secondly, we will want to calculate the proportions of each flat type in a block. We are assuming that blocks with higher proportions of greater than four room will fetch a higher resale value.

In [8]:
# Using a for-loop to calculuate proportions for the following columns

# col = ['one_room_unit', 'two_room_unit', 'three_room_unit', 'four_room_unit', 'five_room_unit', 'exec_unit',
#         'multigen_unit', 'studio_apartment']

# Create empty lists
prop_one_room = []
prop_two_room = []
prop_three_room = []
prop_four_room = []
prop_five_room = []
prop_exec = []
prop_multigen = []
prop_studio_apt = []

# Using a for-loop to calculate the proportions and append to the empty list above
for index, room in hdb_df.iterrows():
    # one room resale
    one_room = room['one_room_unit'] / room['total_dwelling_units']  # calculate proportions
    prop_one_room.append(one_room)                                  # append to new list
    
    # two room resale
    two_room = room['two_room_unit'] / room['total_dwelling_units'] 
    prop_two_room.append(two_room)

    # three room resale
    three_room = room['three_room_unit'] / room['total_dwelling_units'] 
    prop_three_room.append(three_room)
   
    # four room resale
    four_room = room['four_room_unit'] / room['total_dwelling_units'] 
    prop_four_room.append(four_room)

    # five room resale
    five_room = room['five_room_unit'] / room['total_dwelling_units'] 
    prop_five_room.append(five_room)

    # exec resale
    exec = room['exec_unit'] / room['total_dwelling_units'] 
    prop_exec.append(exec)
    
    # multigen resale
    multigen = room['multigen_unit'] / room['total_dwelling_units'] 
    prop_multigen.append(multigen)

    # studio apt resale
    studio = room['studio_apartment'] / room['total_dwelling_units'] 
    prop_studio_apt.append(studio)  


# insert new list into df
hdb_df['prop_one_room'] = prop_one_room 
hdb_df['prop_two_room'] = prop_two_room
hdb_df['prop_three_room'] = prop_three_room
hdb_df['prop_four_room'] = prop_four_room
hdb_df['prop_five_room'] = prop_five_room
hdb_df['prop_exec'] = prop_exec
hdb_df['prop_multigen'] = prop_multigen
hdb_df['prop_studio_apt'] = prop_studio_apt


#print(hdb_df.head()) # debug

- Third, we want to group the planning areas into the URA's Master Plan zoning. 

In [9]:
## Develop lists for dictionary reference
north_east=['Seletar','Punggol','Sengkang','Serangoon','Hougang','Ang Mo Kio']
north = ['Yishun','Simpang','Sembawang','Woodlands','Sungei Kadut','Mandai','Lim Chu Kang','Central Water Catchment']
west = ['Western Water Catchment','Choa Chu Kang','Bukit Panjang','Bukit Batok','Tengah','Jurong West','Jurong East','Boon Lay','Pioneer','Clementi','Tuas']
central = ['Bukit Timah','Queenstown','Bukit Merah','Tanglin','Bishan','Novena','Toa Payoh','Geylang','Marine Parade','Kallang', 'Rochor','Downtown Core','Outram','River Valley','Orchard','Newton']
east = ['Bedok','Paya Lebar','Pasir Ris','Tampines','Changi']


# Dictionary for reference
regions = {"North East":north_east, 'North':north,'West' :west,'East':east,'Central': central}

# create a function to locate region
def region_locator(area):
    for name,region in regions.items():
        if area in region:
            return name

# Mapping new column onto dataframe
hdb_df['region'] = hdb_df['planning_area'].map(region_locator)

print(hdb_df.head()) # debug

       id  flat_type  floor_area_sqm  flat_model  resale_price  tranc_year  tranc_month  mid  hdb_age  max_floor_lvl  multistorey_carpark  total_dwelling_units  one_room_unit  two_room_unit  three_room_unit  four_room_unit  five_room_unit  exec_unit  multigen_unit  studio_apartment planning_area  mall_nearest_distance  mall_within_500m  mall_within_1km  mall_within_2km  hawker_nearest_distance  hawker_within_500m  hawker_within_1km  hawker_within_2km  hawker_food_stalls  mrt_nearest_distance     mrt_name  bus_interchange  mrt_interchange  bus_stop_nearest_distance                           pri_sch_name  nearby_top_sch  prop_one_room  prop_two_room  prop_three_room  prop_four_room  prop_five_room  prop_exec  prop_multigen  prop_studio_apt   region
0   88471     4 ROOM            90.0     Model A      680000.0        2016            5   11       15             25                    0                   142              0              0                0              96              46     

- Fourth, we create columns with boolean values by classifying if the resale unit is closest to which line in Singapore.

In [10]:
# Feature engineer mrt

# lists of stations that sit on the respective lines
# interchange stations appear more than 1 list
north_south_line=['Jurong East','Bukit Batok','Bukit Gombak','Choa Chu Kang','Yew Tee','Kranji','Marsiling','Woodlands','Admiralty','Sembawang','Canberra','Yishun','Khatib','Yio Chu Kang','Ang Mo Kio','Bishan','Braddell','Toa Payoh','Novena','Newton','Orchard','Somerset','Dhoby Ghaut','City Hall','Raffles Place','Marina Bay','Marina South Pier']

north_east_line = ['HarbourFront','Outram Park','Chinatown','Clarke Quay','Little India','Farrer Park','Boon Keng','Potong Pasir','Woodleigh','Serangoon','Kovan','Hougang','Buangkok','Hougang','Sengkang','Punggol']

east_west_line = ['Tuas Link','Tuas West Road','Tuas Crescent','Gul Circle','Joo Koon','Pioneer','Boon Lay','Lakeside','Chinese Garden','Jurong East','Clementi','Dover','Bouna Vista','Commonwealth','Queenstown','Redhill','Tiong Bahru','Outram Park','Tanjong Pagar','Shenton Way','Raffles Place','City Hall','Bugis','Lavender','Kallang','Aljunied','Paya Lebar','Eunos','Kembangan','Bedok','Tanah Merah','Simei','Tampines','Pasir Ris','Expo','Changi Airport']

circle_line = ['HabourFront','Telok Blangah','Labrador Park','Pasir Panjang','Haw Par Villa','Kent Ridge','one-north','Bouna Vista','Holland Village','Farrer Road','Botanic Gardens','Caldecott','Marymount','Bishan','Lorong Chuan','Serangoon','Bartley','Tai Seng','MacPerson','Paya Lebar','Dakota','Mountbatten','Stadium','Nicoll Highway','Promenade','Esplanade','Bras Basah','Dhoby Ghaut']

down_town_line = ['Bukit Panjang','Cashew','Hillview','Beauty World','King Albert Park','Sixth Avenue','Tan Kah Kee','Botanic Gardens','Stevens','Newton','Little India','Rochor','Promenade','Bayfront','Downtown','Telok Ayer','Chinatown','Fort Canning','Bencoolen','Jalan Besar','Bendemeer','Geylang Bahru','Matter','MacPherson','Ubi','Kaki Bukit','Bedok North','Bedok Reservoir','Tampines West','Tampines','Tampines East','Upper Changi','Changi Airport']

thomson_east_coast_line =['Woodlands North','Woodlands','Woodlands South','Springleaf','Lentor','Mayflower','Bright Hill','Upper Thomson','Caldecott','Stevens','Napier','Orchard Boulevard','Orchard','Great World','Havelock','Outram Park','Maxwell','Shenton Way','Marina Bay','Gardens by the Bay']

# create a dictionary
mrts = {"North South Line":north_south_line, 'North East line':north_east_line,'East West Line' :east_west_line,'Circle Line':circle_line,'Down Town Line': down_town_line,'Thomson East Coast Line':thomson_east_coast_line}

# create a function to insert boolean values
def test_mrt(row):
        if row in station:
            return 1
        else:
            return 0
        
# insert the values in the dataframe
for name,station in mrts.items():
    hdb_df[f'{name}'] = hdb_df['mrt_name'].apply(test_mrt)


print(hdb_df.head()) # debug

       id  flat_type  floor_area_sqm  flat_model  resale_price  tranc_year  tranc_month  mid  hdb_age  max_floor_lvl  multistorey_carpark  total_dwelling_units  one_room_unit  two_room_unit  three_room_unit  four_room_unit  five_room_unit  exec_unit  multigen_unit  studio_apartment planning_area  mall_nearest_distance  mall_within_500m  mall_within_1km  mall_within_2km  hawker_nearest_distance  hawker_within_500m  hawker_within_1km  hawker_within_2km  hawker_food_stalls  mrt_nearest_distance     mrt_name  bus_interchange  mrt_interchange  bus_stop_nearest_distance                           pri_sch_name  nearby_top_sch  prop_one_room  prop_two_room  prop_three_room  prop_four_room  prop_five_room  prop_exec  prop_multigen  prop_studio_apt   region  North South Line  North East line  East West Line  Circle Line  Down Town Line  Thomson East Coast Line
0   88471     4 ROOM            90.0     Model A      680000.0        2016            5   11       15             25                    0 

- Fifth, we calculate the number of flats in each floor and obtain the floor density. 

Basically, we want to know the number of units in each floor.

In [11]:
# Calculate the floor density of each row

floor_density = []

for index, room in hdb_df.iterrows():
    temp = math.ceil(room['total_dwelling_units'] / room['max_floor_lvl'])
    floor_density.append(temp)

hdb_df['floor_density'] = floor_density

#print(hdb_df.head()) # debug
    

- Sixth, we create a column that would indicate if the resale unit is on the lower, middle, or upper floors of the block using the mid storey_range that was given. 

In [12]:
# Deriving if the unit is on the lower, middle, or upper floors

floor_category = []
floor_type = []

for index, room in hdb_df.iterrows():
    max_floor = room['max_floor_lvl']

    lower = max_floor//3
    middle = max_floor//3 * 2
    upper = max_floor//3 * 3

    if room['mid'] < lower:
        floor_category.append(1)
        floor_type.append('lower')
    
    elif room['mid'] >= middle:
        floor_category.append(3)
        floor_type.append('upper')

    else:
        floor_category.append(2)
        floor_type.append('middle')


hdb_df['floor_category'] = floor_category
#hdb_df['floor_type'] = floor_type # optional use as an object


In [13]:
# Drop the cols that were used in feature engineering

col = ['total_dwelling_units', 'one_room_unit', 'two_room_unit', 'three_room_unit', 'four_room_unit', 'five_room_unit', 'exec_unit',
        'multigen_unit', 'studio_apartment', 'hawker_food_stalls', 'mid', 'mall_nearest_distance', 'mall_within_500m', 
        'mall_within_1km', 'hawker_nearest_distance', 'hawker_within_500m', 'hawker_within_1km', 'mrt_interchange', 'bus_stop_nearest_distance']


hdb_df = hdb_df.drop(columns=col)

In [14]:
# check final df shape and inspect first 5 rows prior to one-hot encoding
print(hdb_df.shape)
print()
print(hdb_df.head())

(150634, 35)

       id  flat_type  floor_area_sqm  flat_model  resale_price  tranc_year  tranc_month  hdb_age  max_floor_lvl  multistorey_carpark planning_area  mall_within_2km  hawker_within_2km  mrt_nearest_distance     mrt_name  bus_interchange                           pri_sch_name  nearby_top_sch  prop_one_room  prop_two_room  prop_three_room  prop_four_room  prop_five_room  prop_exec  prop_multigen  prop_studio_apt   region  North South Line  North East line  East West Line  Circle Line  Down Town Line  Thomson East Coast Line  floor_density  floor_category
0   88471     4 ROOM            90.0     Model A      680000.0        2016            5       15             25                    0       Kallang                7                 13            330.083069      Kallang                0               geylang methodist school               0            0.0            0.0              0.0        0.676056        0.323944   0.000000            0.0              0.0  Central         

In [15]:
print(hdb_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 150634 entries, 0 to 150633
Data columns (total 35 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       150634 non-null  int64  
 1   flat_type                150634 non-null  object 
 2   floor_area_sqm           150634 non-null  float64
 3   flat_model               150634 non-null  object 
 4   resale_price             150634 non-null  float64
 5   tranc_year               150634 non-null  int64  
 6   tranc_month              150634 non-null  int64  
 7   hdb_age                  150634 non-null  int64  
 8   max_floor_lvl            150634 non-null  int64  
 9   multistorey_carpark      150634 non-null  int64  
 10  planning_area            150634 non-null  object 
 11  mall_within_2km          150634 non-null  int64  
 12  hawker_within_2km        150634 non-null  int64  
 13  mrt_nearest_distance     150634 non-null  float64
 14  mrt_name 

## Creating data dictionary for engineered features


|Feature|Type|Description|
|---|---|---|
|nearby_top_sch|int64|indicates if the resale unit is near a top primary school with a value of 1 else 0|
|prop_one_room|float64|proportion of one room flats in a block|
|prop_two_room|float64|proportion of two room flats in a block|
|prop_three_room|float64|proportion of three room flats in a block|
|prop_four_room|float64|proportion of four room flats in a block|
|prop_exec|float64|proportion of executive flats in a block|
|prop_multigen|float64|proportion of multi-generation flats in a block|
|prop_studio_apt|float64|proportion of studio apartments in a block|
|North South Line|int64|indicates if the resale unit has a mrt on the North-South line in the vicnity with a value of 1 else 0|
|North East Line|int64|indicates if the resale unit has a mrt on the North-East line in the vicnity with a value of 1 else 0|
|East West Line|int64|indicates if the resale unit has a mrt on the East-West line in the vicnity with a value of 1 else 0|
|Circle Line|int64|indicates if the resale unit has a mrt on the Circle line in the vicnity with a value of 1 else 0|
|Down Town Line|int64|indicates if the resale unit has a mrt on the Down Town line in the vicnity with a value of 1 else 0|
|Thomson East Coast Line|int64|indicates if the resale unit has a mrt on the Thomson East Coast Line in the vicnity with a value of 1 else 0|
|floor_density|int64|the number of flats in each floor|
|floor_category|int64|a value of 1 if situated on the lower floors, 2 for middle floors, 3 for upper floors|


In [16]:
# export one csv before dummifying categorical columns
hdb_df.to_csv('../datasets/hdb_no_dummy.csv')

In [17]:
# Dummify the region, flat_type, flat_model columns

hdb_df = pd.get_dummies(data= hdb_df, columns = ['flat_type', 'flat_model', 'region' ], drop_first=True)

In [18]:
# Final inspection of dataframe before exporting to csv
print(hdb_df.shape)
print(hdb_df.head())

(150634, 61)
       id  floor_area_sqm  resale_price  tranc_year  tranc_month  hdb_age  max_floor_lvl  multistorey_carpark planning_area  mall_within_2km  hawker_within_2km  mrt_nearest_distance     mrt_name  bus_interchange                           pri_sch_name  nearby_top_sch  prop_one_room  prop_two_room  prop_three_room  prop_four_room  prop_five_room  prop_exec  prop_multigen  prop_studio_apt  North South Line  North East line  East West Line  Circle Line  Down Town Line  Thomson East Coast Line  floor_density  floor_category  flat_type_2 ROOM  flat_type_3 ROOM  flat_type_4 ROOM  flat_type_5 ROOM  flat_type_EXECUTIVE  flat_type_MULTI-GENERATION  flat_model_Adjoined flat  flat_model_Apartment  flat_model_DBSS  flat_model_Improved  flat_model_Improved-Maisonette  flat_model_Maisonette  flat_model_Model A  flat_model_Model A-Maisonette  flat_model_Model A2  flat_model_Multi Generation  flat_model_New Generation  flat_model_Premium Apartment  flat_model_Premium Apartment Loft  flat_m

In [19]:
# Export to cleaned dataframe to csv

hdb_df.to_csv('../datasets/hdb_cleaned2.csv')